## Player Count

* Display the total number of players

## Purchasing Analysis (Total)

* Calculate the number of unique items, average price, number of purchases, and total revenue


* Create a summary data frame to hold the results


* Display the summary data frame

## Gender Demographics

* Calculate the Percentage and Count of Male Players, the Percentage and Count of Female Players, and the Percentage and Count of Other / Non-Disclosed


* Create a summary data frame to hold the results


* Display the summary data frame

## Purchasing Analysis (Gender)

* Calculate the purchase count, average purchase price, total purchase value, and average total purchase per person by gender


* Create a summary data frame to hold the results


* Display the summary data frame

## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins


* Calculate the total count and percentage of players by age group


* Create a summary data frame to hold the results


* Display Age Demographics Table

## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Calculate the purchase count, average purchase price, total purchase value, and average total purchase per person


* Create a summary data frame to hold the results


* Display the summary data frame

## Top Spenders

* Calculate the purchase count, average purchase price, and total purchase value by SN


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Display a preview of the summary data frame

## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Calculate the purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Display a preview of the summary data frame

## Most Profitable Items

* Sort the Most Popular Items table by total purchase value in descending order


* Display a preview of the data frame